<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Utility-functions" data-toc-modified-id="Utility-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Utility functions</a></span></li><li><span><a href="#Regression-Logic" data-toc-modified-id="Regression-Logic-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regression Logic</a></span></li><li><span><a href="#US,-Fama-French-factors" data-toc-modified-id="US,-Fama-French-factors-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>US, Fama-French factors</a></span></li><li><span><a href="#Developed-ex-US,-Fama-French-factors" data-toc-modified-id="Developed-ex-US,-Fama-French-factors-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Developed ex-US, Fama-French factors</a></span></li><li><span><a href="#US,-QMJ-instead-of-RMW-and-CMA" data-toc-modified-id="US,-QMJ-instead-of-RMW-and-CMA-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>US, QMJ instead of RMW and CMA</a></span></li><li><span><a href="#US,-q-factors" data-toc-modified-id="US,-q-factors-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>US, q-factors</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import statsmodels.api as sm
import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir = "..\\data"

# Utility functions

In [3]:
def read_ff(filename):
    """
    Read ff portfolios
    """
    df = pd.read_excel(os.path.join(data_dir, filename), na_values=[-99.99])
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.to_period('M')
    df = df.set_index('Date')
    df = df.loc['1963-07':'2019-12']
    return df

def read_qmj(filename):
    """
    Read QMJ (AQR factor)
    """
    df = pd.read_excel(os.path.join(data_dir, filename))
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y').dt.to_period('M')
    df = df.set_index('Date')
    df = df.loc['1963-01':'2019-12']
    return df

def read_qfactor(filename):
    """
    Read Q-Factors
    """
    df = pd.read_excel(os.path.join(data_dir, filename))
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y').dt.to_period('M')
    df = df.set_index('Date')
    df = df.loc['1967-01':'2019-12']
    return df[['IA', 'ROE', 'EG']]

def read_qfactor_sorts(filename):
    """
    Read Q-Factors multi sort portfolios
    """
    df = pd.read_excel(os.path.join(data_dir, filename))
    df['Date'] = pd.to_datetime(df['year'].astype(str) + df['month'].astype(str).apply(lambda x: x.zfill(2)), format='%Y%m').dt.to_period('M')
    df = df.drop(['year', 'month'], axis=1)
    df = df.set_index('Date')
    df = df.loc['1967-01':'2019-12']
    return df


def create_weighted_hml_factor(df):
    """
    Create weighted hml factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['HML*'] = (0.9*df['BIG HiBM'] + 0.1*df['SMALL HiBM']) - (0.9*df['BIG LoBM'] + 0.1*df['SMALL LoBM'])
    return df[['HML*']]
    

def create_weighted_wml_factor(df):
    """
    Create weighted wml factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['WML*'] = (0.9*df['BIG HiPRIOR'] + 0.1*df['SMALL HiPRIOR']) - (0.9*df['BIG LoPRIOR'] + 0.1*df['SMALL LoPRIOR'])
    return df[['WML*']]


def create_weighted_rmw_factor(df):
    """
    Create weighted rmw factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['RMW*'] = (0.9*df['BIG HiOP'] + 0.1*df['SMALL HiOP']) - (0.9*df['BIG LoOP'] + 0.1*df['SMALL LoOP'])
    return df[['RMW*']]


def create_weighted_cma_factor(df):
    """
    Create weighted cma factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['CMA*'] = (0.9*df['BIG LoINV'] + 0.1*df['SMALL LoINV']) - (0.9*df['BIG HiINV'] + 0.1*df['SMALL HiINV'])
    return df[['CMA*']]
    

def create_weighted_qmj_factor(df):
    """
    Create weighted qmj factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['QMJ*'] = (0.9*df['Big_Large'] + 0.1*df['Small_Large']) - (0.9*df['Big_Low'] + 0.1*df['Small_Low'])
    return df[['QMJ*']]
    
    
def create_weighted_ia_factor(df):
    """
    Create weighted IA factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['rank_IA'] = df['rank_IA'].apply(lambda x: 'rank_IA_{0}'.format(x))
    df['rank_ME'] = df['rank_ME'].apply(lambda x: 'rank_ME_{0}'.format(x))
    df = df.reset_index().groupby(['Date', 'rank_IA', 'rank_ME'])[['ret_vw']].mean().reset_index()

    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_ME', 'rank_IA'])

    # Create long and short legs
    df['IA*'] = (0.9*df[('rank_ME_2', 'rank_IA_1')] + 0.1*df[('rank_ME_1', 'rank_IA_1')]) - (0.9*df[('rank_ME_2', 'rank_IA_3')] + 0.1*df[('rank_ME_1', 'rank_IA_3')])
    df = df[['IA*']]
    df.columns = df.columns.get_level_values(0) # flatten hierarchical columns using top level headers
    return df[['IA*']]

    
def create_long_short_leg_for_ia(df):
    """
    Create long and short leg for ia factor
    """
    df['rank_IA'] = df['rank_IA'].astype(str)
    df = df.reset_index().groupby(['Date', 'rank_IA'])[['ret_vw']].mean().reset_index()
    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_IA'])
    
    # Create long and short legs
    df['IA_L'] = df['1'] - df['2']
    df['IA_S'] = df['2'] - df['3']
    
    return df[['IA_L', 'IA_S']]


def create_weighted_roe_factor(df):
    """
    Create weighted ROE factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['rank_ROE'] = df['rank_ROE'].apply(lambda x: 'rank_ROE_{0}'.format(x))
    df['rank_ME'] = df['rank_ME'].apply(lambda x: 'rank_ME_{0}'.format(x))
    df = df.reset_index().groupby(['Date', 'rank_ROE', 'rank_ME'])[['ret_vw']].mean().reset_index()

    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_ME', 'rank_ROE'])

    # Create long and short legs
    df['ROE*'] = (0.9*df[('rank_ME_2', 'rank_ROE_3')] + 0.1*df[('rank_ME_1', 'rank_ROE_3')]) - (0.9*df[('rank_ME_2', 'rank_ROE_1')] + 0.1*df[('rank_ME_1', 'rank_ROE_1')])
    df = df[['ROE*']]
    df.columns = df.columns.get_level_values(0) # flatten hierarchical columns using top level headers
    return df[['ROE*']]


def create_long_short_leg_for_roe(df):
    """
    Create long and short leg for roe factor
    """
    df['rank_ROE'] = df['rank_ROE'].astype(str)
    df = df.reset_index().groupby(['Date', 'rank_ROE'])[['ret_vw']].mean().reset_index()
    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_ROE'])
    
    # Create long and short legs
    df['ROE_L'] = df['3'] - df['2']
    df['ROE_S'] = df['2'] - df['1']
    
    return df[['ROE_L', 'ROE_S']]
    
    
def create_weighted_eg_factor(df):
    """
    Create weighted EG factor (90% weight to large-caps and 10% weight to small-caps)
    """
    df['rank_EG'] = df['rank_EG'].apply(lambda x: 'rank_EG_{0}'.format(x))
    df['rank_ME'] = df['rank_ME'].apply(lambda x: 'rank_ME_{0}'.format(x))
    df = df.reset_index().groupby(['Date', 'rank_EG', 'rank_ME'])[['ret_vw']].mean().reset_index()

    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_ME', 'rank_EG'])

    # Create long and short legs
    df['EG*'] = (0.9*df[('rank_ME_2', 'rank_EG_3')] + 0.1*df[('rank_ME_1', 'rank_EG_3')]) - (0.9*df[('rank_ME_2', 'rank_EG_1')] + 0.1*df[('rank_ME_1', 'rank_EG_1')])
    df = df[['EG*']]
    df.columns = df.columns.get_level_values(0) # flatten hierarchical columns using top level headers
    return df[['EG*']]
    
    
def create_long_short_leg_for_eg(df):
    """
    Create long and short leg for eg factor
    """
    df['rank_EG'] = df['rank_EG'].astype(str)
    df = df.reset_index().groupby(['Date', 'rank_EG'])[['ret_vw']].mean().reset_index()
    df = pd.pivot_table(df, values='ret_vw', index=['Date'], columns=['rank_EG'])
    
    # Create long and short legs
    df['EG_L'] = df['3'] - df['2']
    df['EG_S'] = df['2'] - df['1']
    
    return df[['EG_L', 'EG_S']]
    
    
def describe(df, n=2):
    print(df.shape)
    display(df.head(n))
    display(df.tail(n))

In [4]:
def highlight_cells(x, names):
    df = x.copy()
    df.loc[:,:] = '' 
    # set particular css
    for name in names:
        df.loc[name, '{0}*'.format(name)] = 'font-weight: bold'
    return df

# Regression Logic

In [5]:
def run_regression(y, X=None):
    """
    Run regression based on X and y
    """
    if X is not None:
        X = sm.add_constant(X) 
    else:
        X = np.ones((len(y), 1))
        
    model = sm.OLS(endog=y, exog=X).fit()
    coeff = model.params.values
    tvalues = model.tvalues.values
    
    if X.shape[1] == 1:
        return '{0:.2f}'.format(coeff[0]), '({0:.2f})'.format(tvalues[0])
    
    return ['{0:.2f}'.format(x) for x in coeff], ['({0:.2f})'.format(x) for x in tvalues]

In [6]:
def get_exhibit(df, x_cols, y_cols, name=''):
    """
    Get exhibit results for a long leg
    """
    # Exhibit template
    exhibit = pd.DataFrame(columns=x_cols,
                           index=pd.MultiIndex.from_product([y_cols, ['coeff.', 't-stat.']], names=[name, '']))
    
    exhibit = exhibit.fillna('-')
    
    for idx, y_col in enumerate(y_cols, 1):
        X = df[[x_col for x_col in x_cols if x_col != 'alpha']]
        X = X.dropna() # drop NaN to avoid error on regression
        y = df.loc[X.index.values[0]:, y_col]
        coeff, tvalues = run_regression(y=y, X=X)
        exhibit.loc[(y_col, 'coeff.'), x_cols] = coeff
        exhibit.loc[(y_col, 't-stat.'), x_cols] = tvalues
        
    return exhibit

# US, Fama-French factors

In [7]:
# FF5 
ff5_us = read_ff("FF5_US.xlsx")

# WML
wml_us = read_ff("WML_US.xlsx")

# Calculate long legs of each factor and join with ff5 dataframe.
# HML
ff_hml_us = read_ff("FF_SMB_HML_US.xlsx")
ff_hml_us = create_weighted_hml_factor(ff_hml_us)

# WML
ff_wml_us = read_ff("FF_SMB_WML_US.xlsx")
ff_wml_us = create_weighted_wml_factor(ff_wml_us)

# RMW
ff_rmw_us = read_ff("FF_SMB_RMW_US.xlsx")
ff_rmw_us = create_weighted_rmw_factor(ff_rmw_us)

# CMA
ff_cma_us = read_ff("FF_SMB_CMA_US.xlsx")
ff_cma_us = create_weighted_cma_factor(ff_cma_us)

# Join with original df
ff5_us = (ff5_us.join(wml_us)
                .join(ff_hml_us)
                .join(ff_wml_us)
                .join(ff_rmw_us)
                .join(ff_cma_us))

describe(ff5_us)

(678, 11)


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,HML*,WML*,RMW*,CMA*
Date,,,,,,,,,,,
1963-07,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,-1.42079,1.543,0.90793,-1.33837
1963-08,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,1.95408,-0.010,0.30292,-0.01701


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,HML*,WML*,RMW*,CMA*
Date,,,,,,,,,,,
2019-11,3.87,0.50,-1.86,-1.50,-1.29,0.12,-2.62,-1.47933,-3.779,-1.14460,-1.10505
2019-12,2.77,0.96,1.83,0.21,1.31,0.14,-2.13,1.60273,-1.234,1.07923,2.01024


In [8]:
exhibit7_us_ff = get_exhibit(df=ff5_us,
                             x_cols=['alpha', 'Mkt-RF', 'SMB', 'HML*', 'WML*', 'RMW*', 'CMA*'], 
                             y_cols=['HML', 'WML', 'RMW', 'CMA'],
                             name='US, Fama-French factors')

exhibit7_us_ff.style.apply(highlight_cells, names=['HML', 'WML', 'RMW', 'CMA'], axis=None)

# Developed ex-US, Fama-French factors

In [9]:
# FF5 
ff5_developed_exus = read_ff("FF5_Developed_ExUS.xlsx")

# WML
wml_developed_exus = read_ff("WML_Developed_ExUS.xlsx")

# Calculate long legs of each factor and join with ff5 dataframe.
# HML
ff_hml_developed_exus = read_ff("FF_SMB_HML_Developed_ExUS.xlsx")
ff_hml_developed_exus = create_weighted_hml_factor(ff_hml_developed_exus)

# WML
ff_wml_developed_exus = read_ff("FF_SMB_WML_Developed_ExUS.xlsx")
ff_wml_developed_exus = create_weighted_wml_factor(ff_wml_developed_exus)

# RMW
ff_rmw_developed_exus = read_ff("FF_SMB_RMW_Developed_ExUS.xlsx")
ff_rmw_developed_exus = create_weighted_rmw_factor(ff_rmw_developed_exus)

# CMA
ff_cma_developed_exus = read_ff("FF_SMB_CMA_Developed_ExUS.xlsx")
ff_cma_developed_exus = create_weighted_cma_factor(ff_cma_developed_exus)

# Join with original df
ff5_developed_exus = (ff5_developed_exus.join(wml_developed_exus)
                                        .join(ff_hml_developed_exus)
                                        .join(ff_wml_developed_exus)
                                        .join(ff_rmw_developed_exus)
                                        .join(ff_cma_developed_exus))

describe(ff5_developed_exus)

(354, 11)


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,HML*,WML*,RMW*,CMA*
Date,,,,,,,,,,,
1990-07,1.99,2.63,0.38,-0.56,0.58,0.68,NaN,1.291,NaN,-1.887,1.036
1990-08,-11.15,-0.62,0.27,1.09,0.01,0.66,NaN,-0.162,NaN,1.137,-0.350


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,HML*,WML*,RMW*,CMA*
Date,,,,,,,,,,,
2019-11,1.34,0.40,-1.49,0.87,-1.19,0.12,0.00,-1.378,-0.369,0.302,-1.231
2019-12,3.39,1.47,0.74,-0.58,-0.27,0.14,0.74,0.940,0.620,-0.931,-0.197


In [10]:
exhibit7_developed_exus_ff = get_exhibit(df=ff5_developed_exus,
                                         x_cols=['alpha', 'Mkt-RF', 'SMB', 'HML*', 'WML*', 'RMW*', 'CMA*'], 
                                         y_cols=['HML', 'WML', 'RMW', 'CMA'],
                                         name='Developed ex US, Fama-French factors')

exhibit7_developed_exus_ff.style.apply(highlight_cells, names=['HML', 'WML', 'RMW', 'CMA'], axis=None)

# US, QMJ instead of RMW and CMA

In [11]:
# FF5 
ff5_us = read_ff("FF5_US.xlsx")

# WML
wml_us = read_ff("WML_US.xlsx")

# QMJ
qmj_us = read_qmj("QMJ_US.xlsx")

# Calculate long legs of each factor and join with ff5 dataframe.
# HML
ff_hml_us = read_ff("FF_SMB_HML_US.xlsx")
ff_hml_us = create_weighted_hml_factor(ff_hml_us)

# WML
ff_wml_us = read_ff("FF_SMB_WML_US.xlsx")
ff_wml_us = create_weighted_wml_factor(ff_wml_us)

# RMW
ff_qmj_us = read_qmj("QMJ_SMB_QMJ_US.xlsx")
ff_qmj_us = create_weighted_qmj_factor(ff_qmj_us)

# Join with original df
ff5_us = (ff5_us.join(wml_us)
                .join(qmj_us)
                .join(ff_hml_us)
                .join(ff_wml_us)
                .join(ff_qmj_us))

describe(ff5_us)

(678, 11)


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,QMJ,HML*,WML*,QMJ*
Date,,,,,,,,,,,
1963-07,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,0.684436,-1.42079,1.543,0.938991
1963-08,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,-0.991462,1.95408,-0.010,-1.036144


,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,QMJ,HML*,WML*,QMJ*
Date,,,,,,,,,,,
2019-11,3.87,0.50,-1.86,-1.50,-1.29,0.12,-2.62,-1.824954,-1.47933,-3.779,0.602661
2019-12,2.77,0.96,1.83,0.21,1.31,0.14,-2.13,-2.387856,1.60273,-1.234,-0.371839


In [12]:
exhibit7_us_qmj = get_exhibit(df=ff5_us,
                             x_cols=['alpha', 'Mkt-RF', 'SMB', 'HML*', 'WML*', 'QMJ*'], 
                             y_cols=['HML', 'WML', 'QMJ'],
                             name='US, QMJ instead of RMW and CMA')

exhibit7_us_qmj.style.apply(highlight_cells, names=['HML', 'WML', 'QMJ'], axis=None)

# US, q-factors

In [13]:
# FF5 
ff5_us = read_ff("FF5_US.xlsx")
ff5_us = ff5_us.loc['1967-01':'2019-12']
ff5_us = ff5_us.drop(['HML', 'RMW', 'CMA'], axis=1) # drop unnecessary columns

# Q-factor of IA
qfactor_us = read_qfactor("QFactor_US.xlsx")

# ---------------------------- Q-factors -------------------------- #
# IA
qfactor_ia_us = read_qfactor_sorts("QFactor_ME_IA_ROE_US.xlsx")
qfactor_ia_us = create_weighted_ia_factor(qfactor_ia_us)

# ROE
qfactor_roe_us = read_qfactor_sorts("QFactor_ME_IA_ROE_US.xlsx")
qfactor_roe_us = create_weighted_roe_factor(qfactor_roe_us)

# EG
qfactor_eg_us = read_qfactor_sorts("QFactor_ME_EG_US.xlsx")
qfactor_eg_us = create_weighted_eg_factor(qfactor_eg_us)

# Join with original df
ff5_us = (ff5_us.join(qfactor_us)
                .join(qfactor_ia_us)
                .join(qfactor_roe_us)
                .join(qfactor_eg_us))

describe(ff5_us)

(636, 9)


,Mkt-RF,SMB,RF,IA,ROE,EG,IA*,ROE*,EG*
Date,,,,,,,,,
1967-01,8.15,9.00,0.43,-2.9434,1.8866,-2.2469,-2.498253,0.690923,-0.96538
1967-02,0.78,3.03,0.36,-0.2064,3.5317,2.5812,-2.016290,2.958040,3.46808


,Mkt-RF,SMB,RF,IA,ROE,EG,IA*,ROE*,EG*
Date,,,,,,,,,
2019-11,3.87,0.50,0.12,-1.0127,-1.1667,-0.3332,-1.117957,-0.764507,0.00409
2019-12,2.77,0.96,0.14,1.8360,-1.3200,-0.7981,2.647177,1.298727,1.28574


In [14]:
exhibit7_us_qfactor = get_exhibit(df=ff5_us,
                                  x_cols=['alpha', 'Mkt-RF', 'SMB', 'IA*', 'ROE*', 'EG*'], 
                                  y_cols=['IA', 'ROE', 'EG'],
                                  name='US, q-factors')

exhibit7_us_qfactor.style.apply(highlight_cells, names=['IA', 'ROE', 'EG'], axis=None)